# **BLADEI: Bitstream-Level Abnormality Detection for Embedded Inference**
### *Copyright (c) 2025, Rye Stahle-Smith* 
---
#### ***Description:***
#### `deploy_model.py` — On-Device Inference (PYNQ-Z1)

This script runs real-time classification of FPGA bitstreams on the PYNQ-Z1 using the pre-trained model and TSVD transformer.

#### ***Features:***
- Loads `.bit` files from local storage  
- Extracts sparse and structural features  
- Applies TSVD transformation  
- Predicts class (`Benign`, `Malicious`, or `Empty`) using the trained model  
- Displays prediction result with latency breakdown:
  - Load time  
  - Feature extraction time  
  - Inference time

---

In [1]:
# --------------------------
# Step 1: Deploy Trained Model on PYNQ-Z1
# This step loads the serialized ML model and uses it to make predictions on the PYNQ-Z1 board.
# --------------------------
from model_components.rf_predictor import predict_bitstream
from collections import Counter
import numpy as np
import random
import platform
import psutil
import os
import time

base_path = "trusthub_bitstreams"
categories = ["Empty", "Benign", "Malicious"]
total_time_ms = 0
num_trials = 5

bitstream_files = []
for category in categories:
    folder_path = os.path.join(base_path, category)
    for file in os.listdir(folder_path):
        if file.endswith(".bit"):
            bitstream_files.append(os.path.join(folder_path, file))

label_map = {
    0: "Empty (Class 0)",
    1: "Benign AES (Class 1)",
    2: "Benign RS232 (Class 2)",
    3: "Malicious AES (Class 3)",
    4: "Malicious RS232 (Class 4)"
}

for trial in range(num_trials):
    bitstream_path = random.choice(bitstream_files)
    filename = os.path.basename(bitstream_path)
    folder = os.path.basename(os.path.dirname(bitstream_path))

    print(f"*** Trial {trial + 1}: Processing {filename}... ***")

    if folder == "Empty":
        actual_class = "Empty (Class 0)"
    elif folder == "Benign":
        if filename.startswith("AES"):
            actual_class = "Benign AES (Class 1)"
        elif filename.startswith("RS232"):
            actual_class = "Benign RS232 (Class 2)"
        else:
            actual_class = "Benign (Unknown Class)"
    elif folder == "Malicious":
        if filename.startswith("AES"):
            actual_class = "Malicious AES (Class 3)"
        elif filename.startswith("RS232"):
            actual_class = "Malicious RS232 (Class 4)"
        else:
            actual_class = "Malicious (Unknown Class)"
    else:
        actual_class = "Unknown"

    # Measure Load Time
    start_load = time.time()
    with open(bitstream_path, 'rb') as f:
        data = f.read()
    end_load = time.time()

    # Measure Feature Extraction Time
    start_feat = time.time()
    size = len(data)
    if size == 0:
        features = np.zeros(256)
    else:
        counts = Counter(data)
        dense_vec = np.zeros(256)
        for byte_val, count in counts.items():
            dense_vec[byte_val] = count / size
        features = dense_vec
    end_feat = time.time()

    # Measure Prediction Time
    start_pred = time.time()
    prediction = predict_bitstream(features)
    end_pred = time.time()

    predicted_class = label_map.get(prediction, "Unknown")

    print(f"Actual Class:    {actual_class}")
    print(f"Predicted Class: {predicted_class}")

    load_time_ms = (end_load - start_load) * 1000
    feat_time_ms = (end_feat - start_feat) * 1000
    pred_time_ms = (end_pred - start_pred) * 1000

    print(f"\n=== Latency Summary ===")
    print(f"Load Bitstream:      {load_time_ms:.2f} ms")
    print(f"Feature Extraction:  {feat_time_ms:.2f} ms")
    print(f"Prediction:          {pred_time_ms:.2f} ms\n")
    
    total_time_ms += load_time_ms + feat_time_ms + pred_time_ms
    
print(f"\nAverage Latency: {total_time_ms / num_trials / 1000:.2f} s")

*** Trial 1: Processing empty5.bit... ***
Actual Class:    Empty (Class 0)
Predicted Class: Empty (Class 0)

=== Latency Summary ===
Load Bitstream:      185.72 ms
Feature Extraction:  3111.52 ms
Prediction:          17.27 ms

*** Trial 2: Processing RS232_T1400_Trojan.bit... ***
Actual Class:    Malicious RS232 (Class 4)
Predicted Class: Malicious RS232 (Class 4)

=== Latency Summary ===
Load Bitstream:      188.69 ms
Feature Extraction:  3110.94 ms
Prediction:          17.08 ms

*** Trial 3: Processing RS232_T700.bit... ***
Actual Class:    Benign RS232 (Class 2)
Predicted Class: Benign RS232 (Class 2)

=== Latency Summary ===
Load Bitstream:      187.14 ms
Feature Extraction:  3109.11 ms
Prediction:          16.52 ms

*** Trial 4: Processing RS232_T700_Trojan.bit... ***
Actual Class:    Malicious RS232 (Class 4)
Predicted Class: Malicious RS232 (Class 4)

=== Latency Summary ===
Load Bitstream:      187.39 ms
Feature Extraction:  3109.43 ms
Prediction:          17.36 ms

*** Trial 5

In [2]:
# --------------------------
# Step 2: List Device and CPU Information
# This step gathers and prints system and processor information from the PYNQ-Z1 to help verify the hardware environment.
# --------------------------
print("=== System Information: ===")
print("System:", platform.system())
print("Node Name:", platform.node())
print("Release:", platform.release())
print("Version:", platform.version())
print("Machine:", platform.machine())
print("Processor:", platform.processor())

print("\n=== CPU Information: ===")
print("CPU Cores:", psutil.cpu_count(logical=False))
print("Logical Processors:", psutil.cpu_count(logical=True))
print("CPU Usage per Core:", psutil.cpu_percent(percpu=True))
print("Total RAM:", psutil.virtual_memory().total / 1024**2, "MB")

=== System Information: ===
System: Linux
Node Name: pynq
Release: 5.15.19-xilinx-v2022.1
Version: #1 SMP PREEMPT Mon Apr 11 17:52:14 UTC 2022
Machine: armv7l
Processor: armv7l

=== CPU Information: ===
CPU Cores: 2
Logical Processors: 2
CPU Usage per Core: [8.5, 95.6]
Total RAM: 493.58203125 MB
